Test 1 (SAND) GRU model

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout,Activation
from keras import optimizers
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
sand = 'https://raw.githubusercontent.com/loguansiang/fyp/main/SAND%20fyp.csv'
# slp= 'https://raw.githubusercontent.com/loguansiang/fyp/main/SLP%20fyp.csv'
# mana = 'https://raw.githubusercontent.com/loguansiang/fyp/main/MANA%20fyp.csv'

df = pd.read_csv(sand)

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-14,0.086577,0.086577,0.059597,0.065645,0.065645,118988161
2020-08-15,0.066077,0.069509,0.048985,0.050964,0.050964,36213441
2020-08-16,0.051438,0.069508,0.048064,0.060847,0.060847,33023213
2020-08-17,0.060847,0.065073,0.052525,0.053999,0.053999,11686493
2020-08-18,0.053784,0.060990,0.050675,0.051700,0.051700,7401293
...,...,...,...,...,...,...
2022-04-13,2.781983,2.886945,2.751847,2.869923,2.869923,321784424
2022-04-14,2.869366,2.916186,2.714203,2.780882,2.780882,349909147
2022-04-15,2.781060,2.820187,2.755915,2.803639,2.803639,189256519


In [ ]:
pip install ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import ta
from ta import add_all_ta_features
from ta.utils import dropna

In [ ]:
df['Pre_Close'] = df['Close'].shift(+1)
df[('H-L')] = df['High'] - df['Low']
df[('0-C')]= df['Open'] - df['Close']
df['SMA_7'] = df.Close.rolling(7, min_periods=1).mean()
df['rsi_7']=ta.momentum.rsi(df.Close,window=7)
#df['MACD'] = ta.trend.macd_diff(df.Close)
df['EMA_7']= df['Close'].ewm(span=7).mean()
df = df.drop(columns=['Adj Close'])
df.dropna(inplace= True)
df

,Open,High,Low,Close,Volume,Pre_Close,H-L,0-C,SMA_7,rsi_7,EMA_7
Date,,,,,,,,,,,
2020-08-20,0.052095,0.057325,0.050800,0.053648,7332475,0.052095,0.006525,-0.001553,0.055557,36.094661,0.054128
2020-08-21,0.053644,0.053738,0.044235,0.044235,5121097,0.053648,0.009503,0.009409,0.052498,23.308807,0.051380
2020-08-22,0.044395,0.049011,0.040692,0.046011,4347394,0.044235,0.008319,-0.001616,0.051791,28.856159,0.049928
2020-08-23,0.045990,0.048237,0.041965,0.044007,3114426,0.046011,0.006272,0.001983,0.049385,26.347291,0.048360
2020-08-24,0.043902,0.053543,0.042178,0.051299,7868186,0.044007,0.011365,-0.007397,0.048999,46.203257,0.049127
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-13,2.781983,2.886945,2.751847,2.869923,321784424,2.782421,0.135098,-0.087940,2.886363,40.236566,2.909637
2022-04-14,2.869366,2.916186,2.714203,2.780882,349909147,2.869923,0.201983,0.088484,2.843185,36.188051,2.877448
2022-04-15,2.781060,2.820187,2.755915,2.803639,189256519,2.780882,0.064272,-0.022579,2.829312,38.046757,2.858996


In [ ]:
def data_prep(dataset, target, data_start, data_end, look_back, horizon):
    dataX,dataY = [],[]
    data_start = data_start + look_back
    if data_end is None:
        data_end = len(dataset) - horizon

    for i in range(data_start, data_end):
        a = range(i-look_back, i)
        dataX.append(dataset[a])

        b = range(i+1, i+1+horizon)
        dataY.append(target[b])
    return np.array(dataX), np.array(dataY)

In [ ]:
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
# X_data = X_scaler.fit_transform(df[['Open', 'High', 'Low','Volume','H-L',	'0-C','SMA_7','rsi_7','EMA_7']])//experiment4
X_data = X_scaler.fit_transform(df[['Open', 'High', 'Low','Volume']])
Y_data = Y_scaler.fit_transform(df[['Close']])

##['Open', 'High', 'Low','Volume']
##['H-L',	'0-C','SMA_7','rsi_7','EMA_7']

In [ ]:
##TRAIN_SPLIT = int(len(df) * 0.8)
look_back = 10
horizon = 10
train_split = int(len(df) * 0.8)
x_train, y_train = data_prep(X_data, Y_data, 0, train_split, look_back, horizon)
x_vali, y_vali = data_prep(X_data, Y_data, train_split, None, look_back, horizon)

In [ ]:
x_train

array([[[2.83694734e-03, 3.03201955e-03, 3.16017951e-03, 4.25735345e-04],
        [3.04350110e-03, 2.60551547e-03, 2.21115712e-03, 2.26069081e-04],
        [1.81017909e-03, 2.04346228e-03, 1.69898860e-03, 1.56211107e-04],
        ...,
        [3.66729615e-03, 5.14159175e-03, 3.69012865e-03, 2.89300352e-03],
        [5.78670583e-03, 6.00244749e-03, 5.82336908e-03, 2.07498525e-03],
        [5.17891237e-03, 5.06585071e-03, 5.34285873e-03, 7.81648200e-04]],

       [[3.04350110e-03, 2.60551547e-03, 2.21115712e-03, 2.26069081e-04],
        [1.81017909e-03, 2.04346228e-03, 1.69898860e-03, 1.56211107e-04],
        [2.02286680e-03, 1.95143157e-03, 1.88301076e-03, 4.48858994e-05],
        ...,
        [5.78670583e-03, 6.00244749e-03, 5.82336908e-03, 2.07498525e-03],
        [5.17891237e-03, 5.06585071e-03, 5.34285873e-03, 7.81648200e-04],
        [5.60895491e-03, 5.00806398e-03, 5.17546072e-03, 6.14401888e-04]],

       [[1.81017909e-03, 2.04346228e-03, 1.69898860e-03, 1.56211107e-04],
        

In [ ]:
print ('Multiple window of past history\n')
print(x_train[0])
print ('\n Target horizon\n')
print (y_train[0])

Multiple window of past history

[[2.83694734e-03 3.03201955e-03 3.16017951e-03 4.25735345e-04]
 [3.04350110e-03 2.60551547e-03 2.21115712e-03 2.26069081e-04]
 [1.81017909e-03 2.04346228e-03 1.69898860e-03 1.56211107e-04]
 [2.02286680e-03 1.95143157e-03 1.88301076e-03 4.48858994e-05]
 [1.74443925e-03 2.58232944e-03 1.91380159e-03 4.74104920e-04]
 [2.67693150e-03 2.97506515e-03 2.43290889e-03 2.91205679e-04]
 [2.30156103e-03 3.20668766e-03 2.62892935e-03 5.16636406e-04]
 [3.66729615e-03 5.14159175e-03 3.69012865e-03 2.89300352e-03]
 [5.78670583e-03 6.00244749e-03 5.82336908e-03 2.07498525e-03]
 [5.17891237e-03 5.06585071e-03 5.34285873e-03 7.81648200e-04]]

 Target horizon

[[0.0047021 ]
 [0.00347374]
 [0.00337472]
 [0.00161431]
 [0.00174523]
 [0.00080584]
 [0.00120685]
 [0.00084849]
 [0.0022731 ]
 [0.00209141]]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evalution
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

import tensorflow as tf

In [ ]:
batch_size = 32
buffer_size = 18

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.batch(batch_size).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_vali, y_vali))
val_data = val_data.batch(batch_size).repeat()

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
# from keras.layers.merge import concatenate
import tensorflow as tf

In [ ]:
x_train.shape[-2:]

(10, 4)

In [ ]:
'''
# define model
import tensorflow as tf
from tensorflow.keras import layers
# Set random seed for as reproducible results as possible
tf.random.set_seed(7)
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[-2:])))
cnn_model.add(Conv1D(filters=64, kernel_size=1))
cnn_model.add(MaxPooling1D(pool_size=(1)))
cnn_model.add(Conv1D(filters=128, kernel_size=1))
cnn_model.add(Flatten())
cnn_model.add(Dense(256,))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(10))
cnn_model.compile(optimizer='adam', loss='mse')
cnn_model.summary()
'''

"\n# define model\nimport tensorflow as tf\nfrom tensorflow.keras import layers\n# Set random seed for as reproducible results as possible\ntf.random.set_seed(7)\ncnn_model = Sequential()\ncnn_model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(x_train.shape[-2:])))\ncnn_model.add(Conv1D(filters=64, kernel_size=1))\ncnn_model.add(MaxPooling1D(pool_size=(1)))\ncnn_model.add(Conv1D(filters=128, kernel_size=1))\ncnn_model.add(Flatten())\ncnn_model.add(Dense(256,))\ncnn_model.add(Dropout(0.2))\ncnn_model.add(Dense(10))\ncnn_model.compile(optimizer='adam', loss='mse')\ncnn_model.summary()\n"

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.optimizers import Adam
# Set the learning rate
#0.0001,0.001,0.01,0.1,1
learning_rate = 0.001

# Create the optimizer with the desired learning rate
optimizer = Adam(learning_rate=learning_rate)
# Set random seed for as reproducible results as possible
tf.random.set_seed(7)

lstm_model = Sequential()
lstm_model.add(LSTM(100,return_sequences=True,input_shape=(x_train.shape[-2:])))
lstm_model.add(Dense(20,activation='linear'))
lstm_model.add(LSTM(50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=horizon,activation='linear'))
lstm_model.add(Activation('linear'))
lstm_model.compile(optimizer=optimizer, loss='mse')
lstm_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 10, 100)           42000     
                                                                 
 dense_2 (Dense)             (None, 10, 20)            2020      
                                                                 
 lstm_3 (LSTM)               (None, 50)                14200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 10)                510       
                                                                 
 activation_1 (Activation)   (None, 10)                0         
                                                                 
Total params: 58,730
Trainable params: 58,730
Non-trai

In [ ]:
#model_path = 'CNN_Multivariate.h5'
#early_stopings = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min')
#checkpoint =  tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
#callbacks=[early_stopings,checkpoint]

In [ ]:
history = lstm_model.fit(train_data,epochs=50,steps_per_epoch=100,validation_data=val_data,validation_steps=50,verbose=1)#callback
##history = cnn_model.fit(train_data,epochs=50,steps_per_epoch=100,validation_data=val_data,validation_steps=50,verbose=1)

Epoch 1/50
100/100 [==============================] - 7s 32ms/step - loss: 0.0164 - val_loss: 0.0108
Epoch 2/50
100/100 [==============================] - 4s 36ms/step - loss: 0.0072 - val_loss: 0.0204
Epoch 3/50
100/100 [==============================] - 4s 37ms/step - loss: 0.0060 - val_loss: 0.0178
Epoch 4/50
100/100 [==============================] - 4s 44ms/step - loss: 0.0047 - val_loss: 0.0288
Epoch 5/50
100/100 [==============================] - 4s 36ms/step - loss: 0.0050 - val_loss: 0.0361
Epoch 6/50
100/100 [==============================] - 4s 38ms/step - loss: 0.0048 - val_loss: 0.0450
Epoch 7/50
100/100 [==============================] - 4s 41ms/step - loss: 0.0036 - val_loss: 0.0556
Epoch 8/50
100/100 [==============================] - 4s 38ms/step - loss: 0.0031 - val_loss: 0.0723
Epoch 9/50
100/100 [==============================] - 4s 40ms/step - loss: 0.0034 - val_loss: 0.0562
Epoch 10/50
100/100 [==============================] - 3s 25ms/step - loss: 0.0023 - val_lo

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'validation loss'])
plt.show()

In [ ]:
df.columns

In [ ]:
##['Open', 'High', 'Low','Volume']
##['H-L',	'0-C','SMA_7','rsi_7','EMA_7']

# data_val = X_scaler.fit_transform(df[['Open', 'High', 'Low','Volume','H-L',	'0-C','SMA_7','rsi_7','EMA_7']].tail(10))
data_val = X_scaler.fit_transform(df[['Open', 'High', 'Low','Volume']].tail(10))
# data_val = X_scaler.fit_transform(df[['Open', 'High', 'Low','Volume']].tail(10))
val_rescaled = data_val.reshape(1, data_val.shape[0], data_val.shape[1])
pred=lstm_model.predict(val_rescaled)
pred_Inverse = Y_scaler.inverse_transform(pred)
pred_Inverse

In [ ]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')


In [ ]:
validate = (df[['Close']].tail(10))
timeseries_evaluation_metrics_func(validate['Close'],pred_Inverse[0])

In [ ]:
plt.figure(figsize=(16,9))
plt.plot( list(validate['Close']))
plt.plot( list(pred_Inverse[0]))
plt.title("Actual vs Predicted")
plt.ylabel("Close")
plt.legend(('Actual','predicted'))
plt.show()